# 查找插件
可以通过插件搜索工具PlugFinder来查找插件，也是ChatGPT的插件，可以在插件商店搜索并安装
使用PlugFinder搜索你想调用的插件，例如PDF插件，安装后，可以在ChatGPT中调用

可以参考下面分享的链接
https://chat.openai.com/share/97e3e3d8-722e-4caf-9f36-bb5359c21147

点击插件的名字，就可以跳转到插件的定义，如AskYourPdf，定义地址是：https://plugin.askyourpdf.com/.well-known/openapi.yaml
需要检查插件是否有用户授权，授权的暂时无法调用
定义在：https://plugin.askyourpdf.com/.well-known/ai-plugin.json,
"auth": {
    "type": "none"
  },
如果是none，代表没有授权，可以直接调用

# 插件API对应的函数生成
函数可以让ChatGPT帮我们生成，具体可参考下面下面分享的链接
https://chat.openai.com/share/50d8c3ea-a6d3-4e40-8ca0-08ccc2e18bdb

In [ ]:
import requests

def send_request(method, url, data=None, params=None):
    response = requests.request(method, url, json=data, params=params)
    response.raise_for_status()
    return response.json()

# Base URL
BASE_URL = "https://plugin.askyourpdf.com"  # 请将这个URL替换为实际的API URL

# 用来上传pdf到向量数据库，以便后面查询
def download_pdf(url):
    path = "/api/download_pdf"
    full_url = BASE_URL + path
    params = {"url": url}
    response = send_request("POST", full_url, params=params)
    return response

# 用来查询pdf的内容，需要指定doc_id和查询内容
def perform_query(doc_id, query):
    path = "/query"
    full_url = BASE_URL + path
    data = {"doc_id": doc_id, "query": query}
    response = send_request("POST", full_url, data=data)
    return response

In [ ]:
# 调用上传pdf的例子
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
response = download_pdf(pdf_url)
print(response)

In [ ]:
# 调用查询pdf的例子
doc_id = "dff4ba5d-0c37-43b8-81bf-44ddf7fddf2a"  # 替换为实际的文档ID
query = "What is the Self-attention?"  # 替换为实际的查询内容
response = perform_query(doc_id, query)

print(response)

# 下面演示API调用

In [ ]:
import openai
import json

openai.api_key = "sk-xxx"  # 替换为实际的API Key
# Step 1, 先上传要对话的pdf文件
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "I want to upload pdf use this url:https://arxiv.org/pdf/1706.03762.pdf"}],
        functions=[
            {
                "name": "download_pdf",
                "description": "upload a pdf",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "url": {"type": "string","description": "The pdf url"},
                    },
                    "required": ["url"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, 检查函数调用
    if message.get("function_call"):
        # Step 3, 调用函数
        function_call = message.get('function_call')
        arguments = json.loads(function_call.get('arguments'))
        function_response = download_pdf(url=arguments.get("url"))

        # Step 4, 调用查询函数
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "I want to search content with pdf, The query content is What is the Self-attention?,The doc id is " + function_response["docId"]}
            ],
            functions=[
            {
                "name": "perform_query",
                "description": "search content with a pdf",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "doc_id": {"type": "string","description": "The doc id"},
                        "query": {"type": "string","description": "The query content"},
                    },
                    "required": ["doc_id","query"],
                },
            }
        ],
        function_call="auto",
        )

        message = second_response["choices"][0]["message"]
        # Step 5, 检查函数调用
        if message.get("function_call"):
            # Step 6, 调用函数
            function_call = message.get('function_call')
            arguments = json.loads(function_call.get('arguments'))
            function_response = perform_query(doc_id=arguments.get("doc_id"),query=arguments.get("query"))

            return function_response

print(run_conversation())